In [1]:
import pandas as pd

In [160]:
import pandas as pd

trips = pd.DataFrame({
    "id": [1,2,3,4,5,6,7,8,9,10],
    "client_id": [1,2,3,4,1,2,3,2,3,4],
    "driver_id": [10,11,12,13,10,11,12,12,10,13],
    "city_id": [1,1,6,6,1,6,12,12,12,12],
    "status": [
        "completed", "cancelled_by_driver", "completed", "cancelled_by_client",
        "completed", "completed", "completed", "completed", "completed", "cancelled_by_driver"
    ],
    "request_at": [
        "2013-10-01","2013-10-01","2013-10-01","2013-10-01",
        "2013-10-02","2013-10-02","2013-10-02",
        "2013-10-03","2013-10-03","2013-10-03"
    ]
})

trips["request_at"] = pd.to_datetime(trips["request_at"])
trips


,id,client_id,driver_id,city_id,status,request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,12,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [98]:
users = pd.DataFrame({
    "users_id": [1,2,3,4,10,11,12,13],
    "banned": ["No","Yes","No","No","No","No","No","No"],
    "role": ["client","client","client","client","driver","driver","driver","driver"]
})

users

,users_id,banned,role
0,1,No,client
1,2,Yes,client
2,3,No,client
3,4,No,client
4,10,No,driver
5,11,No,driver
6,12,No,driver
7,13,No,driver


In [99]:
mask = (users.role == "client")

In [100]:
clients = users[mask]
clients

,users_id,banned,role
0,1,No,client
1,2,Yes,client
2,3,No,client
3,4,No,client


In [105]:
merged1 = pd.merge(trips, clients, left_on = "client_id", right_on = "users_id", how = "left")
merged1

,id,client_id,driver_id,city_id,status,request_at,users_id,banned,role
0,1,1,10,1,completed,2013-10-01,1,No,client
1,2,2,11,1,cancelled_by_driver,2013-10-01,2,Yes,client
2,3,3,12,6,completed,2013-10-01,3,No,client
3,4,4,13,6,cancelled_by_client,2013-10-01,4,No,client
4,5,1,10,1,completed,2013-10-02,1,No,client
5,6,2,11,6,completed,2013-10-02,2,Yes,client
6,7,3,12,12,completed,2013-10-02,3,No,client
7,8,2,12,12,completed,2013-10-03,2,Yes,client
8,9,3,10,12,completed,2013-10-03,3,No,client
9,10,4,13,12,cancelled_by_driver,2013-10-03,4,No,client


In [106]:
mask = (users.role == "driver")

In [107]:
drivers = users[mask]
drivers

,users_id,banned,role
4,10,No,driver
5,11,No,driver
6,12,No,driver
7,13,No,driver


In [110]:
merged2 = pd.merge(merged1, drivers, left_on = "driver_id", right_on = "users_id", how = "left", suffixes=("_client", "_driver"))
merged2

,id,client_id,driver_id,city_id,status,request_at,users_id_client,banned_client,role_client,users_id_driver,banned_driver,role_driver
0,1,1,10,1,completed,2013-10-01,1,No,client,10,No,driver
1,2,2,11,1,cancelled_by_driver,2013-10-01,2,Yes,client,11,No,driver
2,3,3,12,6,completed,2013-10-01,3,No,client,12,No,driver
3,4,4,13,6,cancelled_by_client,2013-10-01,4,No,client,13,No,driver
4,5,1,10,1,completed,2013-10-02,1,No,client,10,No,driver
5,6,2,11,6,completed,2013-10-02,2,Yes,client,11,No,driver
6,7,3,12,12,completed,2013-10-02,3,No,client,12,No,driver
7,8,2,12,12,completed,2013-10-03,2,Yes,client,12,No,driver
8,9,3,10,12,completed,2013-10-03,3,No,client,10,No,driver
9,10,4,13,12,cancelled_by_driver,2013-10-03,4,No,client,13,No,driver


In [152]:
df = merged2.groupby(by = "request_at").apply(lambda x: ((x["status"].str.startswith("cancelled")) & (x["banned_client"] == "No")).sum()/x["banned_client"].str.startswith("No").sum()).rename("Cancellation Rate").reset_index()
df

,request_at,Cancellation Rate
0,2013-10-01,0.333333
1,2013-10-02,0.000000
2,2013-10-03,0.500000


In [153]:
df = df.rename(columns = {"request_at":"Day"})
df

,Day,Cancellation Rate
0,2013-10-01,0.333333
1,2013-10-02,0.000000
2,2013-10-03,0.500000


In [154]:
df["Cancellation Rate"] = round(df["Cancellation Rate"], 2)
df

,Day,Cancellation Rate
0,2013-10-01,0.33
1,2013-10-02,0.00
2,2013-10-03,0.50


In [162]:
import pandas as pd

mask = (
    (trips.request_at > pd.to_datetime("2013-10-01")) &
    (trips.request_at < pd.to_datetime("2013-10-03"))
)

if mask.any():
    print("Yes")


Yes
